In [1]:
import argparse
import open3d as o3d
import torch
import numpy as np
from easydict import EasyDict as edict
from torch.utils.data import DataLoader

import lib.utils as lu
# from kitti import KITTIDataset
from temp_dataset import KITTIDataset

# parser = argparse.ArgumentParser()
# parser.add_argument(
#     "--eps", type=float, default=1.0, help="Sets the distance threshold = 10 + eps to compute overlapping points"
# )
# args = parser.parse_args()

config = edict(lu.load_config("configs/kitti.yaml"))


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [16]:
# get pair of point clouds from the dataset
dataset = KITTIDataset(split="train", data_augmentation=False, config=config)
loader = DataLoader(dataset, batch_size=128, shuffle=False, drop_last=True)

Num_train: 4541


In [17]:
len(loader)

35

In [10]:
# load inlier coordinates
src_pcd, tgt_pcd, rot, trans = dataset.__getitem__(0)

ValueError: too many values to unpack (expected 4)

In [ ]:
src_pcd.shape

In [ ]:
src, tgt = torch.Tensor(src_pcd).cuda(), torch.Tensor(tgt_pcd).cuda()

In [ ]:
src.device

In [ ]:
# src, tgt = src[-5000:, :], tgt[-5000:, :]
rot, trans = torch.Tensor(rot).cuda(), torch.Tensor(trans).cuda()
src_t = rot @ src.T + trans

In [ ]:
l2_distances = torch.cdist(src_t.T, tgt, 2, compute_mode="donot_use_mm_for_euclid_dist")

In [ ]:
src, tgt, src_t = src.cpu().numpy(), tgt.cpu().numpy(), src_t.cpu().numpy()

In [ ]:
src_t

In [ ]:
src_p = o3d.geometry.PointCloud()
src_p.points = o3d.utility.Vector3dVector(src)
src_p.paint_uniform_color([1, 0, 0])

src_tp = o3d.geometry.PointCloud()
src_tp.points = o3d.utility.Vector3dVector(src_t.T)
src_tp.paint_uniform_color([0, 0, 1])

tgt_p = o3d.geometry.PointCloud()
tgt_p.points = o3d.utility.Vector3dVector(tgt)
tgt_p.paint_uniform_color([0, 1, 0])



In [ ]:
torch.min(l2_distances)

In [ ]:
a = l2_distances.cpu().numpy() < 1

In [ ]:
np.unique(a, return_index=True, return_counts=True)

In [ ]:
a

In [ ]:
# o3d.visualization.draw_geometries([src_p]) # see source & target point clouds

In [ ]:
o3d.visualization.draw_geometries([tgt_p, src_p]) # see source & target point clouds


In [ ]:
o3d.visualization.draw_geometries([tgt_p, src_tp]) # see transformed source & target point clouds

In [ ]:
o3d.visualization.draw_geometries([tgt_p, src_tp, src_p]) # see source, transformed source  & target point clouds

In [ ]:
tgt.shape

In [ ]:
a = l2_distances < 0.5

In [ ]:
y = a.nonzero()

In [ ]:
x = torch.zeros_like(torch.Tensor(src))

In [ ]:
x = x[:, 0]

In [ ]:
x[y[:, 0]] = 1

In [ ]:
x.nonzero().shape

In [ ]:
a[:, 0]